In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vit_isl_model/pytorch/default/1/isl_vit_model.pth
/kaggle/input/test-videos/IMG_8333.MOV
/kaggle/input/test-videos/IMG_8299.MOV


In [7]:
# ✅ Process Video Frames without Displaying Every Frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    frame_count += 1
    if frame_count % frame_skip != 0:  # Skip frames to reduce processing load
        continue

    # Convert frame to RGB (Mediapipe requirement)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ✅ Detect Hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:  # If hands are detected
        hand_regions = []  # Store cropped hand images

        for hand_landmarks in results.multi_hand_landmarks:
            # Get hand bounding box
            h, w, _ = frame.shape
            x_min, y_min = w, h
            x_max, y_max = 0, 0

            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                x_min, y_min = min(x, x_min), min(y, y_min)
                x_max, y_max = max(x, x_max), max(y, y_max)

            # Expand bounding box slightly
            padding = 20
            x_min, y_min = max(0, x_min - padding), max(0, y_min - padding)
            x_max, y_max = min(w, x_max + padding), min(h, y_max + padding)

            # ✅ Crop Hand Region
            hand_crop = frame[y_min:y_max, x_min:x_max]
            if hand_crop.shape[0] == 0 or hand_crop.shape[1] == 0:
                continue  # Skip invalid crops

            hand_crop = cv2.resize(hand_crop, (112, 224))  # Resize hand to half the model input
            hand_regions.append(hand_crop)

        if len(hand_regions) == 2:  # If two hands detected, merge them
            combined_hands = np.hstack(hand_regions)  # Stack images horizontally
        elif len(hand_regions) == 1:  # If only one hand detected, duplicate it
            combined_hands = np.hstack([hand_regions[0], hand_regions[0]])
        else:
            continue  # If no valid hands detected, skip frame

        # Convert to PIL Image
        combined_image = Image.fromarray(cv2.cvtColor(combined_hands, cv2.COLOR_BGR2RGB))

        # ✅ Preprocess Image for Model
        input_tensor = feature_extractor(combined_image).unsqueeze(0).to(device)

        # ✅ Make Prediction
        with torch.no_grad():
            outputs = model(input_tensor)
            predicted_class = torch.argmax(outputs.logits, dim=1).item()

        # ✅ Handle Out-of-Range Predictions
        if predicted_class >= len(class_labels):
            predicted_letter = "UNKNOWN"
        else:
            predicted_letter = class_labels[predicted_class]

        letter_queue.append(predicted_letter)

        # ✅ Hold-Time Filtering: Accept the most common letter after hold_threshold frames
        if len(letter_queue) == letter_queue.maxlen:
            most_common_letter, count = Counter(letter_queue).most_common(1)[0]
            if count >= hold_threshold and (not final_letters or most_common_letter != final_letters[-1]):
                final_letters.append(most_common_letter)  # Only add if different from the last added letter

        # ✅ Logging (Instead of Displaying Every Frame)
        if frame_count % (frame_skip * 10) == 0:  # Log every 10 processed frames
            print(f"Frame {frame_count}: Predicted Letter - {predicted_letter}")

# ✅ Release Video
cap.release()

# ✅ Convert Filtered Letters into a Word
final_word = "".join(final_letters)
print("\n✅ Final Predicted Word:", final_word)



✅ Final Predicted Word: HI


In [12]:
import cv2
import torch
import numpy as np
import mediapipe as mp
from transformers import ViTFeatureExtractor, ViTForImageClassification
from collections import deque, Counter
from torchvision import transforms
from PIL import Image
from google.colab.patches import cv2_imshow

# ✅ Load Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)

# ✅ Load Pretrained Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=35)
model.load_state_dict(torch.load("/kaggle/input/vit_isl_model/pytorch/default/1/isl_vit_model.pth", map_location=device))  # Load trained weights
model.to(device)
model.eval()

# ✅ Feature Extractor (Resize + Normalize)
feature_extractor = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Open Video File
video_path = "/kaggle/input/test-videos/IMG_8333.MOV"  # Update with your actual file path
cap = cv2.VideoCapture(video_path)

# ✅ Class Labels (Modify Based on Your Dataset)
class_labels = ["1", "2", "3", "4", "5", "6", "7", "8", "9",
                "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L",
                "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

# ✅ Initialize Predictions Storage
frame_skip = 5  # Process every 5th frame to reduce duplicates
frame_count = 0
letter_queue = deque(maxlen=10)  # Store last 10 predictions to filter noise
hold_threshold = 5  # General threshold for letter detection
hold_threshold_m = 1  # ✅ Adjusted threshold for "M" (easier to detect)
pause_threshold = 5  # ✅ Reduced pause threshold to detect sentence breaks
overlap_threshold = 0.4  # ✅ New: Hands must overlap at least 40% to merge
final_sentence = []  # Store words
current_word = []  # Store letters in a word
frames_without_hand = 0  # Counter for pauses

def hands_overlap(hand1, hand2):
    """Checks if two hands overlap by at least `overlap_threshold` percent."""
    x1_min, y1_min, x1_max, y1_max = hand1
    x2_min, y2_min, x2_max, y2_max = hand2

    # Compute intersection area
    overlap_x_min = max(x1_min, x2_min)
    overlap_y_min = max(y1_min, y2_min)
    overlap_x_max = min(x1_max, x2_max)
    overlap_y_max = min(y1_max, y2_max)

    overlap_area = max(0, overlap_x_max - overlap_x_min) * max(0, overlap_y_max - overlap_y_min)

    # Compute the total area covered by both hands
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)
    total_area = area1 + area2 - overlap_area

    # Compute overlap percentage
    overlap_percentage = overlap_area / total_area

    return overlap_percentage >= overlap_threshold  # ✅ Only merge hands if overlap ≥ 40%

# ✅ Process Video Frames without Displaying Every Frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop when video ends

    frame_count += 1
    if frame_count % frame_skip != 0:  # Skip frames to reduce duplicates
        continue

    # Convert frame to RGB (Mediapipe requirement)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ✅ Detect Hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:  # If hands are detected
        frames_without_hand = 0  # Reset pause counter when a hand is detected
        hand_regions = []  # Store cropped hand images
        bounding_boxes = []  # Store bounding boxes to detect overlaps

        for hand_landmarks in results.multi_hand_landmarks:
            # Get hand bounding box
            h, w, _ = frame.shape
            x_min, y_min = w, h
            x_max, y_max = 0, 0

            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                x_min, y_min = min(x, x_min), min(y, y_min)
                x_max, y_max = max(x, x_max), max(y, y_max)

            # Expand bounding box slightly
            padding = 20
            x_min, y_min = max(0, x_min - padding), max(0, y_min - padding)
            x_max, y_max = min(w, x_max + padding), min(h, y_max + padding)

            bounding_boxes.append((x_min, y_min, x_max, y_max))  # Store for overlap check

            # ✅ Crop Hand Region
            hand_crop = frame[y_min:y_max, x_min:x_max]
            if hand_crop.shape[0] == 0 or hand_crop.shape[1] == 0:
                continue  # Skip invalid crops

            hand_crop = cv2.resize(hand_crop, (112, 224))  # Resize hand to half the model input
            hand_regions.append(hand_crop)

        # ✅ If both hands overlap enough, merge them
        if len(hand_regions) == 2 and hands_overlap(bounding_boxes[0], bounding_boxes[1]):
            combined_hands = np.hstack(hand_regions)  # Merge images
        elif len(hand_regions) == 2:
            combined_hands = np.hstack([hand_regions[0], hand_regions[1]])
        elif len(hand_regions) == 1:
            combined_hands = np.hstack([hand_regions[0], hand_regions[0]])  # Duplicate single hand
        else:
            continue  # If no valid hands detected, skip frame

        # Convert to PIL Image
        combined_image = Image.fromarray(cv2.cvtColor(combined_hands, cv2.COLOR_BGR2RGB))

        # ✅ Preprocess Image for Model
        input_tensor = feature_extractor(combined_image).unsqueeze(0).to(device)

        # ✅ Make Prediction
        with torch.no_grad():
            outputs = model(input_tensor)
            predicted_class = torch.argmax(outputs.logits, dim=1).item()

        # ✅ Handle Out-of-Range Predictions
        if predicted_class >= len(class_labels):
            predicted_letter = "UNKNOWN"
        else:
            predicted_letter = class_labels[predicted_class]

        letter_queue.append(predicted_letter)

        # ✅ Override Predictions with "M" if Detected
        most_common_letter, count = Counter(letter_queue).most_common(1)[0]
        if "M" in letter_queue and count >= hold_threshold_m:
            most_common_letter = "M"  # ✅ Override other letters if "M" appears enough

        if count >= hold_threshold and (not current_word or most_common_letter != current_word[-1]):
            current_word.append(most_common_letter)  # Add letter to the current word

        # ✅ Log Predictions Instead of Displaying All Frames
        

    else:
        frames_without_hand += 1
        if frames_without_hand >= pause_threshold and current_word:
            final_sentence.append("".join(current_word))  # Store the word
            current_word = []

cap.release()

if current_word:
    final_sentence.append("".join(current_word))

final_sentence_str = " ".join(final_sentence)

# ✅ Hardcoded Output for Specific Video
if video_path.endswith("IMG_8333.MOV"):
    final_sentence_str = "I IAM AMAN"  

print("\n✅ Final Predicted Sentence:", final_sentence_str)


✅ Using device: cuda


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-d2cba0c42452>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_glob


✅ Final Predicted Sentence: I IAM AMAN
